In [ ]:
import sys
import os
import numpy as np
import pandas as pd
import transformers
import tensorflow as tf
import matplotlib.pyplot as plt
import datetime
ROOT = os.path.abspath('../')
sys.path.append(ROOT)
import models
from util import *
from sklearn import manifold
import re
import itertools

In [241]:
tokenizer = transformers.AutoTokenizer.from_pretrained('jplu/tf-xlm-roberta-base')

In [377]:
tt = validation_raw.iloc[136].comment_text.encode()
print(str(tt).count('\\x'))
len(tt.decode())



316


1434

In [353]:
tt

b'Selamlar. Vallaha kaynam\xc4\xb1\xc5\x9f yav...  Yer Fener G\xc3\xb6k Cimbom  adl\xc4\xb1 FB-GS ezeli rekabeti anlatan bir kitap var sadece... Ancak piyasada \xc3\xa7e\xc5\x9fitli kitaplar bulabilirsin. En bilineni \xc4\xb0nk\xc4\xb1lap Yay\xc4\xb1nevi nden \xc3\xa7\xc4\xb1kan futbolla ilgili kitaplar... Bu arada Shilton \xc4\xb1 sana ithaf ediyorum  Mrb. Mahtumkulu Firaki maddesi de telifli i\xc3\xa7erikten olu\xc5\x9fuyor. HS \xc5\x9fablonu kald\xc4\xb1r\xc4\xb1ld\xc4\xb1\xc4\x9f\xc4\xb1 i\xc3\xa7in direk size yazmak istedim, kolay gelsin...Rapsar  Merhaba, madde silindi. Kullan\xc4\xb1c\xc4\xb1ya telif i\xc3\xa7in bir uyar\xc4\xb1 \xc5\x9fablonu g\xc3\xb6nderdim. Kolay gelsin...Viki\xc3\xa7izer '

In [21]:
wrong = reduced_train[np.round(train_predict) != y_train[:,None]]

In [385]:
idx = 1837
idx=1975
idx=773
idx=1992
print(wrong.iloc[idx,:].comment_text)
print(wrong.iloc[idx,:].toxic)

" 

 == Anti-catholicism == 

 I just want to let you know that in the ""homosexuality and catholicism"" deletion of quotes by some catholic saints, there was no copyrighted text as you alleged, which makes you a liar and an anti-catholicism troll. 

 I hope you are brave enough to answer this accusation."
0


In [386]:
wrong.iloc[idx,:].comment_text

'" \n\n == Anti-catholicism == \n\n I just want to let you know that in the ""homosexuality and catholicism"" deletion of quotes by some catholic saints, there was no copyrighted text as you alleged, which makes you a liar and an anti-catholicism troll. \n\n I hope you are brave enough to answer this accusation."'

In [387]:
s = wrong.iloc[idx,:].comment_text.encode('utf-8')
s.count(b'\xcc')
s.count(b'\xcd')
s

b'" \n\n == Anti-catholicism == \n\n I just want to let you know that in the ""homosexuality and catholicism"" deletion of quotes by some catholic saints, there was no copyrighted text as you alleged, which makes you a liar and an anti-catholicism troll. \n\n I hope you are brave enough to answer this accusation."'

In [368]:
import re
s1 = re.sub(b'\xcc.',b'',s)
s2 = re.sub(b'\xcd.',b'',s1)
re.sub(b'\xd2\x89',b'',s2).decode()


"Nevertheless, you don't get the running joke, the edits should stay on the user Zalgo. He has chosen this handle and now he must live with Zalgo's wrath, Zalgo doesn't like spindly little faggots running around using Zalgo's name in vain. Zalgo hits back. What is th..th..this.. OH GOD ŅO PLEASE ZĂLGỎ ILL DO ANYŢHING YOÛ WANT JUST  \n\n PLĘAŞE GOD ŅȎ ZALGO ŇO"

In [372]:
s.decode('ascii','ignore').encode('ascii').decode()

"Nevertheless, you don't get the running joke, the edits should stay on the user Zalgo. He has chosen this handle and now he must live with Zalgo's wrath, Zalgo doesn't like spindly little faggots running around using Zalgo's name in vain. Zalgo hits back. What is th..th..this.. OH GOD O PLEASE ZLG ILL DO ANYHING YO WANT JUST  \n\n PLAE GOD  ZALGO O"

In [369]:
tokenizer.convert_ids_to_tokens((tokenizer.encode('PLAE GOD  ZALGO O')))

['<s>', '▁PLA', 'E', '▁GOD', '▁', 'ZAL', 'GO', '▁O', '</s>']

In [297]:
i = b'\xcc\x80'
i.decode()

'̀'

In [256]:
t = b'.thi\xcc\xa9'
b = t.replace(b'\xcc',b'')
t.decode()

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xa9 in position 4: invalid start byte

In [247]:
t = wrong.iloc[idx,:].comment_text.encode().decode(encoding='ascii',errors='ignore')
t

"Nevertheless, you don't get the running joke, the edits should stay on the user Zalgo. He has chosen this handle and now he must live with Zalgo's wrath, Zalgo doesn't like spindly little faggots running around using Zalgo's name in vain. Zalgo hits back. What is th..th..this.. OH GOD O PLEASE ZLG ILL DO ANYHING YO WANT JUST  \n\n PLAE GOD  ZALGO O"

In [383]:
def clean_text(text):
    place_holder = '<placeholder>'
    # remove strange \xcc\.. \xcd\.. etc. character
    # if total number \xcc \xcd exceed certain thresh hold
    # remove all \xc.. by converteing to ascii and back
    n_schar = str(text.encode()).count('\\xc')
    n_char = len(text)
    if n_schar/n_char >= 0.3:
        text = text.encode('ascii','ignore').decode()
    # limit all character repetition to 5
    text=''.join(''.join(i)[:5] for _,i in itertools.groupby(text))
    # remove nextline 
    text = text.replace('\n',' ')
    text = text.replace('""',place_holder)
    # remove " at behining and end
    text = text.replace('"',' ')
    # replace "" with "
    text = text.replace(place_holder,'\"')
    text = text.replace('==',' ')
    text = text.replace('-=',' ')
    text = text.replace('=-',' ')
    # remove ':' at start of sentence
    text = re.sub('^:*','',text)
    return text


In [390]:
clean_text(wrong.iloc[idx,:].comment_text)

'       Anti-catholicism      I just want to let you know that in the "homosexuality and catholicism" deletion of quotes by some catholic saints, there was no copyrighted text as you alleged, which makes you a liar and an anti-catholicism troll.    I hope you are brave enough to answer this accusation. '

In [248]:
tokenizer.convert_ids_to_tokens((tokenizer.encode(wrong.iloc[idx,:].comment_text)))

['<s>',
 '▁Never',
 'the',
 'less',
 ',',
 '▁you',
 '▁don',
 "'",
 't',
 '▁get',
 '▁the',
 '▁running',
 '▁joke',
 ',',
 '▁the',
 '▁edit',
 's',
 '▁should',
 '▁stay',
 '▁on',
 '▁the',
 '▁user',
 '▁Z',
 'algo',
 '.',
 '▁He',
 '▁has',
 '▁chose',
 'n',
 '▁this',
 '▁handle',
 '▁and',
 '▁now',
 '▁he',
 '▁must',
 '▁live',
 '▁with',
 '▁Z',
 'algo',
 "'",
 's',
 '▁w',
 'rath',
 ',',
 '▁Z',
 'algo',
 '▁doesn',
 "'",
 't',
 '▁like',
 '▁spin',
 'd',
 'ly',
 '▁little',
 '▁fag',
 'got',
 's',
 '▁running',
 '▁around',
 '▁using',
 '▁Z',
 'algo',
 "'",
 's',
 '▁name',
 '▁in',
 '▁vain',
 '.',
 '▁Z',
 'algo',
 '▁hit',
 's',
 '▁back',
 '.',
 '▁What',
 '▁is',
 '▁th',
 '.',
 '.',
 'th',
 '.',
 '.',
 'thi',
 '̷',
 '<unk>',
 '̥',
 '̅',
 '̓',
 '̆',
 '̈',
 '́',
 '̔',
 '̂',
 '̂',
 '<unk>',
 '́',
 'ͅ',
 's',
 '<unk>',
 '̔',
 '<unk>',
 '̈',
 '́',
 '<unk>',
 '́',
 '͏',
 '<unk>',
 '̲',
 '<unk>',
 '̩',
 '<unk>',
 '̕',
 '.',
 '<unk>',
 '̇',
 '<unk>',
 '̀',
 '͏',
 '̭',
 '<unk>',
 '̲',
 '<unk>',
 '.',
 '<unk>',
 '̰',
 '

In [249]:
tokenizer.convert_ids_to_tokens((tokenizer.encode(t)))

['<s>',
 '▁Never',
 'the',
 'less',
 ',',
 '▁you',
 '▁don',
 "'",
 't',
 '▁get',
 '▁the',
 '▁running',
 '▁joke',
 ',',
 '▁the',
 '▁edit',
 's',
 '▁should',
 '▁stay',
 '▁on',
 '▁the',
 '▁user',
 '▁Z',
 'algo',
 '.',
 '▁He',
 '▁has',
 '▁chose',
 'n',
 '▁this',
 '▁handle',
 '▁and',
 '▁now',
 '▁he',
 '▁must',
 '▁live',
 '▁with',
 '▁Z',
 'algo',
 "'",
 's',
 '▁w',
 'rath',
 ',',
 '▁Z',
 'algo',
 '▁doesn',
 "'",
 't',
 '▁like',
 '▁spin',
 'd',
 'ly',
 '▁little',
 '▁fag',
 'got',
 's',
 '▁running',
 '▁around',
 '▁using',
 '▁Z',
 'algo',
 "'",
 's',
 '▁name',
 '▁in',
 '▁vain',
 '.',
 '▁Z',
 'algo',
 '▁hit',
 's',
 '▁back',
 '.',
 '▁What',
 '▁is',
 '▁th',
 '.',
 '.',
 'th',
 '.',
 '.',
 'this',
 '.',
 '.',
 '▁OH',
 '▁GOD',
 '▁O',
 '▁',
 'PLE',
 'ASE',
 '▁Z',
 'LG',
 '▁',
 'ILL',
 '▁DO',
 '▁',
 'ANY',
 'HING',
 '▁',
 'YO',
 '▁W',
 'ANT',
 '▁JUST',
 '▁PLA',
 'E',
 '▁GOD',
 '▁',
 'ZAL',
 'GO',
 '▁O',
 '</s>']

In [115]:
 r = tokenizer.encode(wrong.iloc[idx,:].comment_text)

In [116]:
tokenizer.convert_ids_to_tokens(r)

['<s>',
 'Ġ"',
 'Ċ',
 'Ċ',
 'No',
 '!',
 'Ġ',
 'ĠThis',
 'Ġis',
 'Ġa',
 'ĠGROUP',
 'ĠEFF',
 'ORT',
 '!',
 'Ġ',
 'ĠWikipedia',
 'Ġis',
 'Ġa',
 'Ġcollaborative',
 'ĠCOMMUN',
 'ITY',
 'Ġand',
 'Ġthere',
 'Ġare',
 'Ġno',
 'Ġschool',
 'Ġessays',
 'Ġhere',
 '.',
 'Ġ',
 'ĠThe',
 'Ġarticle',
 'Ġneeds',
 'Ġto',
 'Ġbe',
 'Ġmore',
 'Ġprofessional',
 'Ġand',
 'Ġadopt',
 'Ġa',
 'Ġbetter',
 'Ġtitle',
 'Ġbesides',
 'Ġthe',
 'Ġreferences',
 '.',
 'Ġ',
 'ĠThis',
 'Ġis',
 'Ġall',
 'Ġthat',
 'Ġneeds',
 'Ġto',
 'Ġbe',
 'Ġdone',
 ',',
 'Ġso',
 'Ġget',
 'Ġoff',
 'Ġyour',
 'Ġhigh',
 'Ġhorse',
 'Ġand',
 'Ġaccomplish',
 'Ġwhat',
 'Ġyou',
 'Ġwant',
 'Ġto',
 'Ġsee',
 'Ġdone',
 '.',
 'Ġ',
 'ĠIf',
 'Ġyou',
 'Ġhave',
 'Ġthese',
 'Ġgoals',
 ',',
 'Ġthen',
 'Ġput',
 'Ġyourself',
 'Ġto',
 'Ġthe',
 'Ġtest',
 'Ġof',
 'Ġsolving',
 'Ġthis',
 'Ġproblem',
 '.',
 'Ġ',
 'ĠThat',
 "'s",
 'Ġwhat',
 'ĠI',
 'Ġdo',
 'Ġwhenever',
 'Ġsomething',
 'Ġpert',
 'ur',
 'bs',
 'Ġme',
 '.',
 'Ġ',
 'ĠYou',
 "'re",
 'Ġjust',
 'Ġlooking',
 'Ġf

In [121]:
len(r)

201

In [136]:
tokenizer.convert_ids_to_tokens([0, 44, 711, 2])

['<s>', 'ĠâĢ', 'ĺ', '</s>']

In [93]:
tokenizer.tokenize(wrong.iloc[idx,:].comment_text)

['"',
 'Ċ',
 'Ċ',
 'No',
 '!',
 'Ġ',
 'ĠThis',
 'Ġis',
 'Ġa',
 'ĠGROUP',
 'ĠEFF',
 'ORT',
 '!',
 'Ġ',
 'ĠWikipedia',
 'Ġis',
 'Ġa',
 'Ġcollaborative',
 'ĠCOMMUN',
 'ITY',
 'Ġand',
 'Ġthere',
 'Ġare',
 'Ġno',
 'Ġschool',
 'Ġessays',
 'Ġhere',
 '.',
 'Ġ',
 'ĠThe',
 'Ġarticle',
 'Ġneeds',
 'Ġto',
 'Ġbe',
 'Ġmore',
 'Ġprofessional',
 'Ġand',
 'Ġadopt',
 'Ġa',
 'Ġbetter',
 'Ġtitle',
 'Ġbesides',
 'Ġthe',
 'Ġreferences',
 '.',
 'Ġ',
 'ĠThis',
 'Ġis',
 'Ġall',
 'Ġthat',
 'Ġneeds',
 'Ġto',
 'Ġbe',
 'Ġdone',
 ',',
 'Ġso',
 'Ġget',
 'Ġoff',
 'Ġyour',
 'Ġhigh',
 'Ġhorse',
 'Ġand',
 'Ġaccomplish',
 'Ġwhat',
 'Ġyou',
 'Ġwant',
 'Ġto',
 'Ġsee',
 'Ġdone',
 '.',
 'Ġ',
 'ĠIf',
 'Ġyou',
 'Ġhave',
 'Ġthese',
 'Ġgoals',
 ',',
 'Ġthen',
 'Ġput',
 'Ġyourself',
 'Ġto',
 'Ġthe',
 'Ġtest',
 'Ġof',
 'Ġsolving',
 'Ġthis',
 'Ġproblem',
 '.',
 'Ġ',
 'ĠThat',
 "'s",
 'Ġwhat',
 'ĠI',
 'Ġdo',
 'Ġwhenever',
 'Ġsomething',
 'Ġpert',
 'ur',
 'bs',
 'Ġme',
 '.',
 'Ġ',
 'ĠYou',
 "'re",
 'Ġjust',
 'Ġlooking',
 'Ġfor',
 'Ġa